In [34]:
########### SSD512 특징 추출 네트워크(멀티 스케일 특징 맵) 정의
from torch import nn
from collections import OrderedDict


class SSDBackbone(nn.Module):
    def __init__(self, backbone):
        super().__init__()
        layer0 = nn.Sequential(backbone.conv1, backbone.bn1, backbone.relu)
        # ResNet의 4개의 스테이지
        layer1 = backbone.layer1
        layer2 = backbone.layer2
        layer3 = backbone.layer3
        layer4 = backbone.layer4
        
        # ResNet 백본
        self.features   = nn.Sequential(layer0, layer1, layer2, layer3)
        # 특징맵 차원 증가 (512차원으로 증가)
        self.upsampling = nn.Sequential(
            nn.Conv2d(in_channels=256, out_channels=512, kernel_size=1),
            nn.ReLU(inplace=True),
        )
        # 마지막 계층에 연결되어 멀티 스케일 특징 맵을 추출하는 계층들
        self.extra = nn.ModuleList(
            [
                nn.Sequential(
                    layer4,
                    nn.Conv2d(in_channels=512, out_channels=1024, kernel_size=1),
                    nn.ReLU(inplace=True),  # T - 입력텐서의 값을 직접 변경하여 메모리 절약 
                                            # F - 새로운 텐서 생성하여 출력 저장
                ),
                nn.Sequential(
                    nn.Conv2d(1024, 256, kernel_size=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(256, 512, kernel_size=3, padding=1, stride=2),
                    nn.ReLU(inplace=True),
                ),
                nn.Sequential(
                    nn.Conv2d(512, 128, kernel_size=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(128, 256, kernel_size=3, padding=1, stride=2),
                    nn.ReLU(inplace=True),
                ),
                nn.Sequential(
                    nn.Conv2d(256, 128, kernel_size=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(128, 256, kernel_size=3),
                    nn.ReLU(inplace=True),
                ),
                nn.Sequential(
                    nn.Conv2d(256, 128, kernel_size=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(128, 256, kernel_size=3),
                    nn.ReLU(inplace=True),
                ),
                nn.Sequential(
                    nn.Conv2d(256, 128, kernel_size=1),
                    nn.ReLU(inplace=True),
                    nn.Conv2d(128, 256, kernel_size=4),
                    nn.ReLU(inplace=True),
                )
            ]
        )


    def forward(self, x):
        # ResNet 백본
        x = self.features(x)
        # 차원 증가
        output = [self.upsampling(x)]

        # 멀티 스케일 특징 맵 추출
        for block in self.extra:
            x = block(x)
            output.append(x)

        # 클래스 분류 및 박스 회귀 네트워크에 전달될 생성된 특징맵 (upsampling 1 + extra 6 = 7개)
        return OrderedDict([(str(i), v) for i, v in enumerate(output)])

In [35]:
########### SSD512 모델 생성
import torch
from torchvision.models import resnet34
from torchvision.models.detection import ssd
from torchvision.models.detection.anchor_utils import DefaultBoxGenerator


backbone_base    = resnet34(weights="ResNet34_Weights.IMAGENET1K_V1")
backbone         = SSDBackbone(backbone_base)
anchor_generator = DefaultBoxGenerator( # 특징맵이 7개이므로 기본박스의 종횡비와 간격도 7개
    # 기본박스 종횡비(가로:세로)
    aspect_ratios = [[2], [2, 3], [2, 3], [2, 3], [2, 3], [2], [2]], #[2]=1:2, 2:1
    # 원본이미지 대비 기본박스 비율
    scales        = [0.07, 0.15, 0.33, 0.51, 0.69, 0.87, 1.05, 1.20],
    # 기본박스 간격 (다운샘플링 비율)(ex. 512x512 -steps 8-> 특정맵64x64)(셀 하나의 크기)
    steps         = [8, 16, 32, 64, 100, 300, 512],
)

device = "mps" if torch.backends.mps.is_available() and torch.backends.mps.is_built() else "cpu"
model  = ssd.SSD(
    backbone         = backbone,
    anchor_generator = anchor_generator,
    size             = (512, 512),
    num_classes      = 3
).to(device)

In [36]:
######## 데이터세트 클래스 선언(Fast R-CNN과 동일코드)

import os
import torch
from PIL import Image
from pycocotools.coco import COCO
from torch.utils.data import Dataset

# MS COCO 데이터세트는 80개의 클래스로 이루어짐
# 이번 절에서는 개와 고양이 클래스를 소규모로 샘플랑해 실습 진행
class COCODataset(Dataset):
    def __init__(self, root, train, transform=None):
        super().__init__()
        directory = "train" if train else "val"
        # annotation json 파일 경로 설정
        annotations = os.path.join(root, "annotations", f"{directory}_annotations.json")
        
        # 복잡한 주석 정보 쉽게 다룰 수 있도록 설계된 COCO 클래스
        self.coco       = COCO(annotations)
        self.iamge_path = os.path.join(root, directory)
        self.transform  = transform

        self.categories = self._get_categories()
        self.data       = self._load_data()

    def _get_categories(self):
        categories = {0: "background"}
        # cats(categories) 속성에서 카테고리 정보 불러오기
        for category in self.coco.cats.values():
            categories[category["id"]] = category["name"]
        return categories
    
    ### COCO 데이터세트 불러오기
    def _load_data(self):
        data = []
        for _id in self.coco.imgs:
            # 이미지 정보 로드
            file_name  = self.coco.loadImgs(_id)[0]["file_name"]
            # Output: [{'license': 1, 'file_name': '000000495357.jpg', ..., 'id': 495357}]이므로 [0]처리
            image_path = os.path.join(self.iamge_path, file_name)
            image      = Image.open(image_path).convert("RGB")

            # 주석 정보 로드 (카테고리 id, bbox 좌표)
            boxes  = []
            labels = []
            # getAnnIds: 이미지 id 입력받고 어노테이션 id 반환
            anns   = self.coco.loadAnns(self.coco.getAnnIds(_id))
            for ann in anns:
                # x, y는 왼쪽 상단 모서리의 좌표
                x, y, w, h = ann["bbox"]
                
                boxes.append([x, y, x + w, y + h])
                labels.append(ann["category_id"])   # [1,2,1,1,..]

            # 타겟 딕셔너리 생성
            target = {
            "image_id": torch.LongTensor([_id]),
            "boxes"   : torch.FloatTensor(boxes),
            "labels"  : torch.LongTensor(labels)
            }
            # 데이터 리스트에 추가
            data.append([image, target])
        return data

    ### 호출 및 길이 반환 메서드
    def __getitem__(self, index):
        image, target = self.data[index]
        if self.transform:
            image = self.transform(image)
        return image, target

    def __len__(self):
        return len(self.data)

In [37]:
####### 데이터로더(Fast R-CNN과 동일코드)
from torchvision import transforms
from torch.utils.data import DataLoader


def collator(batch):
    return tuple(zip(*batch))

transform = transforms.Compose(
    [
        transforms.PILToTensor(),
        transforms.ConvertImageDtype(dtype=torch.float)
    ]
)

train_dataset = COCODataset("../datasets/coco", train=True, transform=transform)
test_dataset  = COCODataset("../datasets/coco", train=False, transform=transform)

train_dataloader = DataLoader(
    train_dataset, batch_size=4, shuffle=True, drop_last=True, collate_fn=collator
)
test_dataloader = DataLoader(
    test_dataset, batch_size=1, shuffle=True, drop_last=True, collate_fn=collator
)

loading annotations into memory...
Done (t=0.07s)
creating index...
index created!
loading annotations into memory...
Done (t=0.00s)
creating index...
index created!


In [27]:
from torch import optim


params       = [p for p in model.parameters() if p.requires_grad]
optimizer    = optim.SGD(params, lr=0.001, momentum=0.9, weight_decay=0.0005)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [ ]:
for epoch in range(10): 
    cost = 0.0
    for idx, (images, targets) in enumerate(train_dataloader): 
        images  = list(image.to(device) for image in images)
        targets = [{k: v.to(device) for k, v in t.items()} for t in targets] #k:image_id,boxes,labels

        loss_dict = model(images, targets)  # 박스 회귀 손실, 객체 분류 손실
        # loss_dict = {'bbox_regression': ~, 'classification': ~}
        # hard negative mining 구현 X
        losses    = sum(loss for loss in loss_dict.values())
        optimizer.zero_grad()
        losses.backward()
        optimizer.step()

        cost += losses

    lr_scheduler.step()
    cost = cost / len(train_dataloader)
    print(f"Epoch : {epoch+1:4d}, Cost : {cost:.3f}")

Epoch :    1, Cost : 6.216  
Epoch :    2, Cost : 5.149  
Epoch :    3, Cost : 4.602  
Epoch :    4, Cost : 4.182  
Epoch :    5, Cost : 3.816  
Epoch :    6, Cost : 3.183  
Epoch :    7, Cost : 2.955  
Epoch :    8, Cost : 2.811  
Epoch :    9, Cost : 2.709  
Epoch :   10, Cost : 2.582

In [ ]:
######## 모델 추론 및 시각화 (Fast R-CNN과 동일코드)
import numpy as np
from PIL import Image
from matplotlib import pyplot as plt
from torchvision.transforms.functional import to_pil_image

# 사각형과 텍스트를 이미지 위에 그리는 함수
def draw_bbox(ax, box, text, color): 
    ax.add_patch(
        plt.Rectangle(
            # 0,1: 왼쪽위 모서리 x,y / 2,3: 오른쪽아래 모서리 x,y
            xy        = (box[0], box[1]),
            width     = box[2] - box[0],
            height    = box[3] - box[1],
            fill      = False,
            edgecolor = color,
            linewidth = 2,
        )
    )
    ax.annotate(
        text     = text,
        xy       = (box[0] - 5, box[1] - 5),
        color    = color,
        weight   = "bold",
        fontsize = 13,
    )

threshold  = 0.5    # 신뢰도 임계값
categories = test_dataset.categories
with torch.no_grad(): 
    model.eval()
    for images, targets in test_dataloader: 
        images  = [image.to(device) for image in images]
        outputs = model(images)
        
        # 0.5 이상의 신뢰도 가진 것만 필터링
        boxes  = outputs[0]["boxes"].to("cpu").numpy()
        labels = outputs[0]["labels"].to("cpu").numpy()
        scores = outputs[0]["scores"].to("cpu").numpy()
        
        boxes  = boxes[scores >= threshold].astype(np.int32)
        labels = labels[scores >= threshold]
        scores = scores[scores >= threshold]

        fig = plt.figure(figsize=(8, 8))
        ax  = fig.add_subplot(1, 1, 1)
        plt.imshow(to_pil_image(images[0]))

        # 예측 상자 시각화
        for box, label, score in zip(boxes, labels, scores): 
            draw_bbox(ax, box, f"{categories[label]} - {score:.4f}", "red")

        # 실제 객체 상자 시각화
        tboxes  = targets[0]["boxes"].numpy()
        tlabels = targets[0]["labels"].numpy()
        for box, label in zip(tboxes, tlabels): 
            draw_bbox(ax, box, f"{categories[label]}", "blue")
            
        plt.show()

![](../스크린샷%202024-05-24%20오전%201.16.25.png)

In [ ]:
######## 모델 평가(Fast R-CNN과 동일코드)
import numpy as np
from pycocotools.cocoeval import COCOeval


with torch.no_grad(): 
    model.eval()
    coco_detections = []
    for images, targets in test_dataloader: 
        images  = [img.to(device) for img in images]
        outputs = model(images)
        
        # 각 이미지의 타겟(boxes,scores,labels)정보 저장
        for i in range(len(targets)): 
            image_id    = targets[i]["image_id"].data.cpu().numpy().tolist()[0]
            boxes       = outputs[i]["boxes"].data.cpu().numpy()
            boxes[:, 2] = boxes[:, 2] - boxes[:, 0] # w
            boxes[:, 3] = boxes[:, 3] - boxes[:, 1] # h
            scores      = outputs[i]["scores"].data.cpu().numpy()
            labels      = outputs[i]["labels"].data.cpu().numpy()

            # 한 이미지에서 경계 상자(boxes) 여러개인 경우
            for instance_id in range(len(boxes)): 
                box        = boxes[instance_id, :].tolist()
                prediction = np.array(
                    [
                        image_id,
                        box[0],
                        box[1],
                        box[2],
                        box[3],
                        float(scores[instance_id]),
                        int(labels[instance_id]),
                    ]
                )
                coco_detections.append(prediction)

    # detection 결과 저장
    coco_detections = np.asarray(coco_detections)
    # 데이터셋 groundtruth API
    coco_gt         = test_dataloader.dataset.coco      # 데이터셋의 ground truth
    coco_dt         = coco_gt.loadRes(coco_detections)  # detection 결과를 COCO 데이터구조로 로드
    coco_evaluator  = COCOeval(coco_gt, coco_dt, iouType="bbox") # 모델 평가 클래스
    # 모델 평가
    coco_evaluator.evaluate()
    coco_evaluator.accumulate()
    coco_evaluator.summarize()

IoU 임계값과 area(객체의 크기), maxDets(검출할 수 있는 최대 객체수)에 따른 모델의 성능 (높을수록 좋은 성능)

![](../스크린샷%202024-05-24%20오전%201.17.38.png)

In [ ]:
########### 출력 채널 할당 방법
# 가상 이미지를 입력해 특성 맵을 추출하고 각 계층의 출력 채널 수를 반환하는 함수
def retrieve_out_channels(model, size):
    model.eval()
    with torch.no_grad():
        device   = next(model.parameters()).device
        image    = torch.zeros((1, 3, size[1], size[0]), device=device)
        # 특성맵 추출
        features = model(image)
        
        if isinstance(features, torch.Tensor):  # 출력이 텐서라면
            features = OrderedDict([("0", features)])
            # 피처 맵의 크기는 [배치 크기, 채널 수, 높이, 너비] 형태이므로, size(1)을 사용하여 채널 수 추출
        out_channels = [x.size(1) for x in features.values()]

    model.train()
    return out_channels

print(retrieve_out_channels(backbone, (512, 512)))

[512, 1024, 512, 256, 256, 256, 256]
